In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '/home/jw3514/Work/CellType_Psy/src')
from CellType_PSY import *
import scanpy as sc
HGNC, ENSID2Entrez, GeneSymbol2Entrez, Entrez2Symbol = LoadGeneINFO()

In [ ]:
def getBiasesBySTR(STR, dfs):
    biases = []
    for df in dfs:
        bias = df.loc[STR, "EFFECT"]
        biases.append(bias)
    biases = np.array(biases)
    return biases

def AddPvalue(ASD_Bias, SibBiasDFs):
    for CT, row in ASD_Bias.iterrows():
        mat_bias = getBiasesBySTR(CT, SibBiasDFs)
        Z, P = GetPermutationP(mat_bias, row["EFFECT"])
        ASD_Bias.loc[CT, "EFFECT2"] = row["EFFECT"] - np.mean(mat_bias)
        ASD_Bias.loc[CT, "Pvalue"] = P
        ASD_Bias.loc[CT, "Z_Match"] = Z
        ASD_Bias.loc[CT, "Z_Pvalue"] = scipy.stats.norm.sf(abs(Z))
    
    acc, qvalues = stats.multitest.fdrcorrection(ASD_Bias["Pvalue"].values, alpha=0.1,
                                                       method="i")
    print(sum(acc))
    ASD_Bias["qvalues"] = qvalues
    #ASD_Bias.to_csv("../dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.subsib.FDR.csv")

    acc, qvalues = stats.multitest.fdrcorrection(ASD_Bias["Z_Pvalue"].values, alpha=0.1,
                                                   method="i")
    print(sum(acc))
    ASD_Bias["qvalues_ZP"] = qvalues
    return ASD_Bias

def LoadCTRLBiasDFs(CTRL_DIR, n_samples=10000):
    CTRLBiasDFs = []
    for i in range(n_samples):
        try:
            DF = pd.read_csv("{}/cont.bias.{}.csv".format(CTRL_DIR, i), index_col=0)
            CTRLBiasDFs.append(DF)
        except:
            continue
    print(len(CTRLBiasDFs))
    return CTRLBiasDFs

# Bias to SubSib

In [ ]:
ASD_BiasDN = pd.read_csv("dat/Bias/ASD.ClusterV3.top60.DN.UMI.Z2.csv", index_col=0)
ASD_Bias = pd.read_csv("dat/Bias/ASD.ClusterV3.top60.UMI.Z2.csv", index_col=0)
ASD_ClusterV3_Bias160DN = pd.read_csv("dat/Bias/ASD.ClusterV3.top160.DN.UMI.Z2.csv", index_col=0)

In [ ]:
SubSibDIR = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/CTRL/SubSampleSib_w_GeneProb/Cluster_V3_UMI_Z2_DN_V2_zclip3/"
SibBiasDFs_z1clip3 = LoadCTRLBiasDFs(SubSibDIR)

In [ ]:
ASD_BiasDN_z1clip3 = pd.read_csv("dat/Bias/ASD.ClusterV3.top60.UMI.Z2.z1clip3.csv", index_col=0)
ASD_BiasDN_z1clip3_addP = AddPvalue(ASD_BiasDN_z1clip3, SibBiasDFs_z1clip3)
ASD_BiasDN_z1clip3_addP.to_csv("dat/Bias/ASD.ClusterV3.top60.UMI.Z2.z1clip3.addP.csv")

In [ ]:
print(len(ASD_BiasDN_z1clip3_addP[ASD_BiasDN_z1clip3_addP["qvalues"]<0.05]))
print(len(ASD_BiasDN_z1clip3_addP[ASD_BiasDN_z1clip3_addP["qvalues"]<0.1]))

In [ ]:
SubSibDIR = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/CTRL/SubSampleSib_w_GeneProb/Cluster_V3_UMI_Z2/"
SibBiasDFs = []
for i in range(10000):
    try:
        DF = pd.read_csv("{}/cont.bias.{}.csv".format(SubSibDIR, i), index_col=0)
        SibBiasDFs.append(DF)
    except:
        continue

In [ ]:
print(len(SibBiasDFs))

In [ ]:
ASD_Bias_addP_sib = AddPvalue(ASD_Bias, SibBiasDFs)

In [ ]:
ASD_Bias_addP_sib.to_csv("dat/Bias/ASD.ClusterV3.top60.UMI.Z2.noDN.pvalues.csv")

In [ ]:
SubSibDIR_DN = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/CTRL/SubSampleSib_w_GeneProb/Cluster_V3_UMI_Z2_DN/"
SibBiasDFs_DN = []
for i in range(100000):
    try:
        DF = pd.read_csv("{}/cont.bias.{}.csv".format(SubSibDIR_DN, i), index_col=0)
        SibBiasDFs_DN.append(DF)
    except:
        continue
print(len(SibBiasDFs_DN))

In [ ]:
SubSibDIR_DN2 = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/CTRL/SubSampleSib_w_GeneProb/Cluster_V3_UMI_Z2_DN_V2/"
SibBiasDFs_DN2 = []
for i in range(10000):
    try:
        DF = pd.read_csv("{}/cont.bias.{}.csv".format(SubSibDIR_DN2, i), index_col=0)
        SibBiasDFs_DN2.append(DF)
    except:
        continue
print(len(SibBiasDFs_DN2))

In [ ]:
ASD_Bias_addP_DNsib_v2 = AddPvalue(ASD_BiasDN, SibBiasDFs_DN2)

In [ ]:
print(ASD_Bias_addP_DNsib_v2[ASD_Bias_addP_DNsib_v2["qvalues"]<0.05].shape)
print(ASD_Bias_addP_DNsib_v2[ASD_Bias_addP_DNsib_v2["qvalues"]<0.1].shape)

In [ ]:
SubSibDIR_DN160 = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/CTRL/SubSampleSib_w_GeneProb/ASD_160_Cluster_V3_UMI_Z2_DN_V2/"
SibBiasDFs_DN160 = []
for i in range(10000):
    try:
        DF = pd.read_csv("{}/cont.bias.{}.csv".format(SubSibDIR_DN160, i), index_col=0)
        SibBiasDFs_DN160.append(DF)
    except:
        continue
print(len(SibBiasDFs_DN160))

In [ ]:
ASD_Bias_addP_160DN = AddPvalue(ASD_ClusterV3_Bias160DN, SibBiasDFs_DN160)

In [ ]:
print(ASD_Bias_addP_160DN[ASD_Bias_addP_160DN["qvalues"]<0.05].shape)
print(ASD_Bias_addP_160DN[ASD_Bias_addP_160DN["qvalues"]<0.1].shape)

In [ ]:
ASD_Bias_addP_160DN.head(10)

In [ ]:
ASD_Bias_addP_160DN.to_csv("dat/Bias/ASD.ClusterV3.top160.UMI.Z2.pvalues.v2.csv")
ASD_Bias_addP_DNsib_v2.to_csv("dat/Bias/ASD.ClusterV3.top60.UMI.Z2.pvalues.v2.csv")

In [ ]:
56
1070

In [ ]:
ASD_Bias_addP_DNsib = AddPvalue(ASD_BiasDN, SibBiasDFs_DN)

In [ ]:
ASD_Bias_addP_DNsib.head(10)

In [ ]:
for CT, row in ASD_Bias.iterrows():
    mat_bias = getBiasesBySTR(CT, SibBiasDFs)
    Z, P = GetPermutationP(mat_bias, row["EFFECT"])
    ASD_Bias.loc[CT, "EFFECT2"] = row["EFFECT"] - np.mean(mat_bias)
    ASD_Bias.loc[CT, "Pvalue"] = P
    ASD_Bias.loc[CT, "Z_Match"] = Z
    ASD_Bias.loc[CT, "Z_Pvalue"] = scipy.stats.norm.sf(abs(Z))
    
acc, qvalues = stats.multitest.fdrcorrection(ASD_Bias["Pvalue"].values, alpha=0.1,
                                                   method="i")
print(sum(acc))
ASD_Bias["qvalues"] = qvalues
#ASD_Bias.to_csv("../dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.subsib.FDR.csv")

acc, qvalues = stats.multitest.fdrcorrection(ASD_Bias["Z_Pvalue"].values, alpha=0.1,
                                               method="i")
print(sum(acc))
ASD_Bias["qvalues_ZP"] = qvalues

In [ ]:
ASD_Bias.head(10)

In [ ]:
ASD_Bias.to_csv("dat/Bias/ASD.ClusterV3.top60.UMI.Z2.pvalues.csv")

In [ ]:
ASD_Bias.shape

In [ ]:
ASD_Bias.head(50)

In [ ]:
ASD_Bias_nDN = pd.read_csv("dat/Bias/ASD.ClusterV3.Merge.Z2.wo_DN.csv", index_col=0)

In [ ]:
plt.scatter(ASD_Bias["EFFECT"].values, ASD_Bias_nDN["EFFECT"].values)

In [ ]:
CompareCT_ABC(ASD_Bias, ASD_Bias_nDN)

In [ ]:
SubSibDIR2 = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/CTRL/SubSampleSib_w_GeneProb/Subclass_SupterType_Merge/"
SibBiasDFs2 = []
for i in range(10000):
    DF = pd.read_csv("{}/cont.bias.{}.csv".format(SubSibDIR, i), index_col=0)
    SibBiasDFs2.append(DF)

In [ ]:
for CT, row in ASD_Bias_nDN.iterrows():
    mat_bias = getBiasesBySTR(CT, SibBiasDFs)
    Z, P = GetPermutationP(mat_bias, row["EFFECT"])
    ASD_Bias_nDN.loc[CT, "Pvalue"] = P
    ASD_Bias_nDN.loc[CT, "Z_Match"] = Z
    ASD_Bias_nDN.loc[CT, "Z_Pvalue"] = scipy.stats.norm.sf(abs(Z))
    
acc, qvalues = stats.multitest.fdrcorrection(ASD_Bias_nDN["Pvalue"].values, alpha=0.1,
                                                   method="i")
print(sum(acc))
ASD_Bias_nDN["qvalues"] = qvalues

acc, qvalues = stats.multitest.fdrcorrection(ASD_Bias_nDN["Z_Pvalue"].values, alpha=0.1,
                                                   method="i")
print(sum(acc))
ASD_Bias_nDN["qvalues_ZP"] = qvalues

In [ ]:
ASD_Bias_nDN.head(50)

In [ ]:
dat1 = []
dat2 = []
for i in range(1000):
    df1 = SibBiasDFs[i]
    df2 = SibBiasDFs2[i]
    dat1.append(df1.head(50)["EFFECT"].mean())
    dat2.append(df2.head(50)["EFFECT"].mean())

In [ ]:
asd_1 = ASD_Bias.head(50)["EFFECT"].mean()
asd_2 = ASD_Bias_nDN.head(50)["EFFECT"].mean()

In [ ]:
plt.scatter(asd_1, asd_2, color="red", zorder=100)
plt.scatter(dat1, dat2)
plt.grid(True)
plt.plot([0.2, 0.9], [0.2, 0.9], ls="--", color="grey")

In [ ]:
RandGeneDIR = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/CTRL/RandGenes/Subclass_SupterType_Merge_DN/"
RandGeneDFs = []
for i in range(10000):
    DF = pd.read_csv("{}/cont.bias.{}.csv".format(RandGeneDIR, i), index_col=0)
    RandGeneDFs.append(DF)

In [ ]:
RandGeneDIR2 = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/CTRL/RandGenes/Subclass_SupterType_Merge/"
RandGeneDFs2 = []
for i in range(10000):
    DF = pd.read_csv("{}/cont.bias.{}.csv".format(RandGeneDIR2, i), index_col=0)
    RandGeneDFs2.append(DF)

In [ ]:
dat1 = []
dat2 = []
for i in range(1000):
    df1 = RandGeneDFs[i]
    df2 = RandGeneDFs2[i]
    dat1.append(df1.head(50)["EFFECT"].mean())
    dat2.append(df2.head(50)["EFFECT"].mean())

In [ ]:
plt.scatter(asd_1, asd_2, color="red", zorder=100)
plt.scatter(dat1, dat2)
plt.grid(True)
plt.plot([0.2, 0.9], [0.2, 0.9], ls="--", color="grey")

In [ ]:
for CT, row in ASD_Bias_nDN.iterrows():
    mat_bias = getBiasesBySTR(CT, RandGeneDFs2)
    Z, P = GetPermutationP(mat_bias, row["EFFECT"])
    ASD_Bias_nDN.loc[CT, "Pvalue"] = P
    ASD_Bias_nDN.loc[CT, "Z_Match"] = Z
    ASD_Bias_nDN.loc[CT, "Z_Pvalue"] = scipy.stats.norm.sf(abs(Z))
    
acc, qvalues = stats.multitest.fdrcorrection(ASD_Bias_nDN["Pvalue"].values, alpha=0.1,
                                                   method="i")
print(sum(acc))
ASD_Bias_nDN["qvalues"] = qvalues

acc, qvalues = stats.multitest.fdrcorrection(ASD_Bias_nDN["Z_Pvalue"].values, alpha=0.1,
                                                   method="i")
print(sum(acc))
ASD_Bias_nDN["qvalues_ZP"] = qvalues

In [ ]:
ASD_Bias_nDN = ASD_Bias_nDN.sort_values("Pvalue")

In [ ]:
ASD_Bias_nDN.head(50)

### Cluster 5k level

In [ ]:
ASD_Cluster_Bias = pd.read_csv("dat/Bias/ASD.ClusterV3.DN.Z2.csv", index_col=0)

In [ ]:
SubSibDIR_ClusterV3 = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/CTRL/SubSampleSib_w_GeneProb/Cluster_V3_Jun15/"
SibBiasDFs_ClusterV3 = []
for i in range(10000):
    DF = pd.read_csv("{}/cont.bias.{}.csv".format(SubSibDIR_ClusterV3, i), index_col=0)
    SibBiasDFs_ClusterV3.append(DF)

In [ ]:
for CT, row in ASD_Cluster_Bias.iterrows():
    mat_bias = getBiasesBySTR(CT, SibBiasDFs_ClusterV3)
    Z, P = GetPermutationP(mat_bias, row["EFFECT"])
    ASD_Cluster_Bias.loc[CT, "Pvalue"] = P
    ASD_Cluster_Bias.loc[CT, "Z_Match"] = Z
    ASD_Cluster_Bias.loc[CT, "Z_Pvalue"] = scipy.stats.norm.sf(abs(Z))
    
acc, qvalues = stats.multitest.fdrcorrection(ASD_Cluster_Bias["Pvalue"].values, alpha=0.1,
                                                   method="i")
print(sum(acc))
ASD_Cluster_Bias["qvalues"] = qvalues
acc, qvalues = stats.multitest.fdrcorrection(ASD_Cluster_Bias["Z_Pvalue"].values, alpha=0.1,
                                               method="i")
print(sum(acc))
ASD_Cluster_Bias["qvalues_ZP"] = qvalues

In [ ]:
#ASD_Cluster_Bias.head(10)

In [ ]:
ASD_Cluster_Bias.to_csv("dat/Bias/ASD.ClusterV3.DN.Z2.palues.csv")

In [ ]:
Classes = ["01 IT-ET Glut", "06 CTX-CGE GABA", "07 CTX-MGE GABA", "12 HY GABA", "14 HY Glut", "18 TH Glut",
          "09 CNU-LGE GABA"]
Class_Pvals = []
for Class in Classes:
    tmp = ASD_Cluster_Bias[ASD_Cluster_Bias["class_id_label"]==Class]
    Class_Pvals.append(tmp["Pvalue"].values)

In [ ]:
QQplot(Class_Pvals, Classes)

In [ ]:
Classes = ["24 MY Glut", "27 MY GABA", "23 P Glut", "26 P GABA", "28 CB GABA", "29 CB Glut"]
Class_Pvals = []
for Class in Classes:
    tmp = ASD_Cluster_Bias[ASD_Cluster_Bias["class_id_label"]==Class]
    Class_Pvals.append(tmp["Pvalue"].values)
QQplot(Class_Pvals, Classes)

In [ ]:
_Class = ["30 Astro-Epen", "31 OPC-Oligo", "32 OEC", "33 Vascular", "34 Immune"]
tmp = ASD_Cluster_Bias[ASD_Cluster_Bias["class_id_label"].isin(_Class)]
QQplot([tmp["Pvalue"].values], ["Non Neuron"])

In [ ]:
Classes = ["30 Astro-Epen", "31 OPC-Oligo", "32 OEC", "33 Vascular", "34 Immune"]
Class_Pvals = []
for Class in Classes:
    tmp = ASD_Cluster_Bias[ASD_Cluster_Bias["class_id_label"]==Class]
    Class_Pvals.append(tmp["Pvalue"].values)
QQplot(Class_Pvals, Classes)

#### If Sibling use DN

In [ ]:
SubSibDIR_ClusterV3_DN = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/CTRL/SubSampleSib_w_GeneProb/Cluster_V3_Jun17_DN2//"
SibBiasDFs_ClusterV3_DN = []
for i in range(10000):
    DF = pd.read_csv("{}/cont.bias.{}.csv".format(SubSibDIR_ClusterV3_DN, i), index_col=0)
    SibBiasDFs_ClusterV3_DN.append(DF)

In [ ]:
ASD_Cluster_Bias_for_DN = pd.read_csv("dat/Bias/ASD.ClusterV3.DN.Z2.csv", index_col=0)

In [ ]:
for CT, row in ASD_Cluster_Bias_for_DN.iterrows():
    mat_bias = getBiasesBySTR(CT, SibBiasDFs_ClusterV3_DN)
    Z, P = GetPermutationP(mat_bias, row["EFFECT"])
    ASD_Cluster_Bias_for_DN.loc[CT, "Bias_Adj"] = row["EFFECT"] - np.mean(mat_bias)
    ASD_Cluster_Bias_for_DN.loc[CT, "Pvalue"] = P
    ASD_Cluster_Bias_for_DN.loc[CT, "Z_Match"] = Z
    if Z == Z:
        ASD_Cluster_Bias_for_DN.loc[CT, "Z_Pvalue"] = scipy.stats.norm.sf(abs(Z))
    else:
        ASD_Cluster_Bias_for_DN.loc[CT, "Z_Pvalue"] = 1
    
acc, qvalues = stats.multitest.fdrcorrection(ASD_Cluster_Bias_for_DN["Pvalue"].values, alpha=0.1,
                                                   method="i")
print(sum(acc))
ASD_Cluster_Bias_for_DN["qvalues"] = qvalues
acc, qvalues = stats.multitest.fdrcorrection(ASD_Cluster_Bias_for_DN["Z_Pvalue"].values, alpha=0.1,
                                               method="i")
print(sum(acc))
ASD_Cluster_Bias_for_DN["qvalues_ZP"] = qvalues

In [ ]:
ASD_Cluster_Bias_for_DN.head(10)

In [ ]:
ASD_Cluster_Bias_for_DN.to_csv("dat/Bias/ASD.ClusterV3.DN.Z2.Sib.DN.palues.csv")

In [ ]:
ASD_Cluster_Bias_for_DN[ASD_Cluster_Bias_for_DN["qvalues"]<0.05].shape

In [ ]:
Classes = ["01 IT-ET Glut", "06 CTX-CGE GABA", "07 CTX-MGE GABA", "12 HY GABA", "14 HY Glut", "18 TH Glut",
          "09 CNU-LGE GABA"]
Class_Pvals = []
for Class in Classes:
    tmp = ASD_Cluster_Bias_for_DN[ASD_Cluster_Bias_for_DN["class_id_label"]==Class]
    Class_Pvals.append(tmp["Pvalue"].values)
QQplot(Class_Pvals, Classes)

In [ ]:
Classes = ["24 MY Glut", "27 MY GABA", "23 P Glut", "26 P GABA", "28 CB GABA", "29 CB Glut"]
Class_Pvals = []
for Class in Classes:
    tmp = ASD_Cluster_Bias_for_DN[ASD_Cluster_Bias_for_DN["class_id_label"]==Class]
    Class_Pvals.append(tmp["Pvalue"].values)
QQplot(Class_Pvals, Classes)

In [ ]:
1 * (30000)  + 12800

In [ ]:
15000 * 10 / 42800

In [ ]:
640000 / 750

In [ ]:
15000 * 10 / 12800

In [ ]:
15000 * 10 / 12800 * 32

# Old

In [ ]:
ASD_GW = pd.read_csv("../../ASD_Circuits/dat/Unionize_bias/Spark_Meta_EWS.GeneWeight.csv", header=None)
ASD_Genes = ASD_GW[0].values

SCZ_GW = pd.read_csv("../../ASD_Circuits/dat/Unionize_bias/SCZ.top61.gw.csv", header=None)
SCZ_Genes = SCZ_GW[0].values

In [ ]:
Mouse_V2_ExpL = pd.read_csv("./dat/ExpressionMats/Subclass_V2_ExpMat.csv", index_col=0)
Mouse_V3_ExpL = pd.read_csv("./dat/ExpressionMats/Subclass_V3_ExpMat.csv", index_col=0)
Mouse_V2_Z1 = pd.read_csv("./dat/ExpressionMats/Subclass_V2_Z1Mat.csv", index_col=0)
Mouse_V3_Z1 = pd.read_csv("./dat/ExpressionMats/Subclass_V3_Z1Mat.csv", index_col=0)

In [ ]:
#Mouse_V2_Cluster_ExpL = pd.read_csv("./dat/ExpressionMats/Cluster_V2_Z1Mat.csv", index_col=0)
#Mouse_V3_Cluster_ExpL = pd.read_csv("./dat/ExpressionMats/Subclass_V3_ExpMat.csv", index_col=0)
Mouse_V2_Cluster_Z1 = pd.read_csv("./dat/ExpressionMats/Cluster_V2_Z1Mat.csv", index_col=0)
Mouse_V3_Cluster_Z1 = pd.read_csv("./dat/ExpressionMats/Cluster_V3_Z1Mat.csv", index_col=0)

In [ ]:
SCZ_Genes = [x for x in SCZ_Genes if x in Mouse_V3_ExpL.index.values]
ASD_Genes = [x for x in ASD_Genes if x in Mouse_V3_ExpL.index.values]

In [ ]:
CT = "046 Vip Gaba"
#CT = "053 Sst Gaba"

In [ ]:
def test(CT):
    Mouse_SC_Exp = Mouse_V3_ExpL
    Mouse_SC_Z1 = Mouse_V3_Z1

    MeanClassExp = Mouse_SC_Exp.loc[:, CT].mean()
    MeanClassZ1 = Mouse_SC_Z1.loc[:, CT].mean()
    MeanASDExp = Mouse_SC_Exp.loc[ASD_Genes, CT].mean()
    MeanASDZ1 = Mouse_SC_Z1.loc[ASD_Genes, CT].mean()
    MeanSCZExp = Mouse_SC_Exp.loc[SCZ_Genes, CT].mean()
    MeanSCZZ1 = Mouse_SC_Z1.loc[SCZ_Genes, CT].mean()
    return MeanClassZ1, MeanASDZ1, MeanSCZZ1

In [ ]:
print(MeanClassZ1, MeanASDZ1, MeanSCZZ1)

In [ ]:
CT = "046 Vip Gaba"
test(CT)

In [ ]:
CT = "047 Sncg Gaba"
test(CT)

In [ ]:
CT = "049 Lamp5 Gaba"
test(CT)

In [ ]:
CT = "051 Pvalb chandelier Gaba"
test(CT)

In [ ]:
CT = "052 Pvalb Gaba"
test(CT)

In [ ]:
CT = "053 Sst Gaba"
test(CT)

In [ ]:
Mouse_SC_Exp = Mouse_V3_ExpL
Mouse_SC_Z1 = Mouse_V3_Z1

MeanClassExp = Mouse_SC_Exp.loc[:, CT].mean()
MeanClassZ1 = Mouse_SC_Z1.loc[:, CT].mean()
MeanASDExp = Mouse_SC_Exp.loc[ASD_Genes, CT].mean()
MeanASDZ1 = Mouse_SC_Z1.loc[ASD_Genes, CT].mean()
MeanSCZExp = Mouse_SC_Exp.loc[SCZ_Genes, CT].mean()
MeanSCZZ1 = Mouse_SC_Z1.loc[SCZ_Genes, CT].mean()

In [ ]:
CT = "053 Sst Gaba"
CT = "046 Vip Gaba"

TestCellTypes = ["061 STR D1 Gaba", "060 OT D3 Folh1 Gaba", "046 Vip Gaba", "047 Sncg Gaba", 
                "049 Lamp5 Gaba", "051 Pvalb chandelier Gaba", "052 Pvalb Gaba", "053 Sst Gaba"]

for CT in TestCellTypes:
    plt.figure(dpi=120)

    sns.kdeplot(Mouse_SC_Z1.loc[:, CT], label="ALL Genes")
    sns.kdeplot(Mouse_SC_Z1.loc[ASD_Genes, CT], label="ASD Genes")
    sns.kdeplot(Mouse_SC_Z1.loc[SCZ_Genes, CT], label="SCZ Genes")
    plt.xlim(-3, 4)
    plt.legend()


In [ ]:
Human_SC_Exp = pd.read_csv("../dat2/HumanCT.ExpL.log.csv", index_col=0)
Human_SC_Z1 = pd.read_csv("../dat2/Human.CT.AllCell.Z1.Entrez.csv", index_col=0)
Human_SC_Exp.columns = [int(x) for x in Human_SC_Exp.columns.values]
#Human_SC_Z2.columns = [int(x) for x in Human_SC_Z2.columns.values]
Human_SC_Z1.columns = [int(x) for x in Human_SC_Z1.columns.values]

In [ ]:
CT = 247 # MGE sst

In [ ]:
#MeanClassExp = Human_SC_Exp.loc[:, clusters].mean()
#MeanClassZ1 = Human_SC_Z1.loc[:, clusters].mean()
#MeanASDExp = Human_SC_Exp.loc[ASD_Genes, clusters].mean()
#MeanASDZ1 = Human_SC_Z1.loc[ASD_Genes, clusters].mean()
#MeanSCZExp = Human_SC_Exp.loc[SCZ_Genes, clusters].mean()
#MeanSCZZ1 = Human_SC_Z1.loc[SCZ_Genes, clusters].mean()

In [ ]:
TestCellTypes = [207, 209, 240, 241, 242, 247, 276]
Names = ["MSN D2", "MSN D1", "pvalb", "sst 1", "sst 2", "sst 3", "vip"]
for name, CT in zip(Names, TestCellTypes):
    plt.figure(dpi=120)

    sns.kdeplot(Human_SC_Z1.loc[:, CT], label="ALL Genes")
    sns.kdeplot(Human_SC_Z1.loc[ASD_Genes, CT], label="ASD Genes")
    sns.kdeplot(Human_SC_Z1.loc[SCZ_Genes, CT], label="SCZ Genes")

    plt.xlim(-2, 4)
    plt.legend()
    plt.xlabel(name)

In [ ]:

TestCellTypes = range(241, 255, 1) # SST
for CT in TestCellTypes:
    plt.figure(dpi=120)

    sns.kdeplot(Human_SC_Z1.loc[:, CT], label="ALL Genes")
    sns.kdeplot(Human_SC_Z1.loc[ASD_Genes, CT], label="ASD Genes")
    sns.kdeplot(Human_SC_Z1.loc[SCZ_Genes, CT], label="SCZ Genes")

    plt.xlim(-2, 4)
    plt.legend()

In [ ]:
MouseSC_Cluster_Ann = pd.read_excel("../../data/Allen_Mouse_Brain_Cell_Atlas/SuppTables/41586_2023_6812_MOESM8_ESM.xlsx",
                          sheet_name = "cluster_annotation", index_col="cluster_id_label")

In [ ]:
DF_TestCellTypes = MouseSC_Cluster_Ann[MouseSC_Cluster_Ann["subclass_label"]=="Vip Gaba"]

In [ ]:
TestCellTypes = DF_TestCellTypes.index.values

for CT in TestCellTypes:
    plt.figure(dpi=120)

    sns.kdeplot(Mouse_V2_Cluster_Z1.loc[:, CT], label="ALL Genes")
    sns.kdeplot(Mouse_V2_Cluster_Z1.loc[ASD_Genes, CT], label="ASD Genes")
    sns.kdeplot(Mouse_V2_Cluster_Z1.loc[SCZ_Genes, CT], label="SCZ Genes")
    plt.xlim(-3, 4)
    plt.legend()

In [ ]:
DF_TestCellTypes = MouseSC_Cluster_Ann[MouseSC_Cluster_Ann["subclass_label"]=="Sst Gaba"]

In [ ]:
TestCellTypes = DF_TestCellTypes.index.values

for CT in TestCellTypes:
    plt.figure(dpi=120)

    sns.kdeplot(Mouse_V3_Cluster_Z1.loc[:, CT], label="ALL Genes")
    sns.kdeplot(Mouse_V3_Cluster_Z1.loc[ASD_Genes, CT], label="ASD Genes")
    sns.kdeplot(Mouse_V3_Cluster_Z1.loc[SCZ_Genes, CT], label="SCZ Genes")
    plt.xlim(-3, 4)
    plt.legend()